# AnyPPG: Model 2

## 1. Load the model

In [ ]:
# clone directory
!git clone https://github.com/Ngk03/AnyPPG.git

In [ ]:
# change direcory
%cd AnyPPG


In [ ]:
# install dependencies:
!pip install -r requirements.txt


restart session

In [ ]:
%cd /content/AnyPPG
import sys
sys.path.insert(0, '/content/AnyPPG')
from model.net1d import Net1D


In [ ]:
import torch
from model.net1d import Net1D

# AnyPPG encoder configuration
anyppg_cfg = {
    "in_channels": 1,
    "base_filters": 64,
    "ratio": 1.0,
    "filter_list": [64, 160, 160, 400, 400, 1024],
    "m_blocks_list": [2, 2, 2, 3, 3, 1],
    "kernel_size": 3,
    "stride": 2,
    "groups_width": 16,
    "verbose": False,
}

# Initialize encoder
anyppg = Net1D(**anyppg_cfg)

# Load pretrained model weights
ckpt_path = "./checkpoint/anyppg_ckpt.pth"
state_dict = torch.load(ckpt_path, map_location="cpu")
anyppg.load_state_dict(state_dict)

## 2.  import packages and connect to google drive

In [ ]:
!pip install mat73

In [ ]:
!pip install vitaldb

In [ ]:
pip install pytorch-lightning

In [ ]:
import sys
is_colab = 'google.colab' in sys.modules

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import the right packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import einops as eo # Added
import pathlib as pl # Added
import seaborn as sns

from mat73 import loadmat

from torch.utils.data import ConcatDataset

import wfdb
import ast
import pickle
import gzip
import random
import urllib.request
import shutil
import ast
from multiprocessing import Pool, cpu_count
# from tqdm import tqdm # Removed redundant import
import os
import os, urllib.request
import itertools
import matplotlib.pyplot as plt
import gdown
import json
from collections import defaultdict



import matplotlib.cm as cm
from matplotlib import collections  as mc
from matplotlib import animation

%matplotlib inline

from scipy.stats import norm
from scipy.stats import entropy
from scipy.ndimage import uniform_filter1d
from scipy.stats import pearsonr

import scipy.io as sio

from datetime import datetime

#transformer:
from transformers import AutoTokenizer, AutoModel



from PIL import Image
from time import time as timer
# import umap

from IPython.display import HTML
from IPython.display import Audio
import IPython

from functools import partial
from collections import Counter

from tqdm.auto import tqdm # Corrected import to get the callable tqdm function

import torch
from torchvision import datasets, transforms
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
import pytorch_lightning as pl




from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import StratifiedShuffleSplit

import glob
import vitaldb

import shutil





# REGRESSOR

# baseline regressor:

In [ ]:
# Baseline: predict subject mean BMI (or global mean) and compute MAE
windowed_dir = '/content/drive/MyDrive/CAS_FinalProject/Vital_Windowed1_Synch'
full_dataset = PPGBMIDataset(windowed_dir, remove_missing_bmi=True)
mean_bmi = full_dataset.bmi_values.mean()
baseline_mae = np.mean(np.abs(full_dataset.bmi_values - mean_bmi))
print(f"Baseline MAE: {baseline_mae:.2f}")


## load data

In [ ]:
# more efficient data loading LAZY LOADING

class PPGBMIDataset(Dataset):
    """
    Memory-efficient Dataset for PPG signals with BMI classification.
    Uses lazy loading and memory mapping to avoid RAM saturation.
    """
    def __init__(self, data_dir, remove_missing_bmi=True):
        self.data_dir = data_dir
        self.batch_files = sorted(
            glob.glob(os.path.join(data_dir, 'PulseDB_Vital_Batch_*_Windowed_Synch.npz'))
        )

        if len(self.batch_files) == 0:
            raise FileNotFoundError(f"No windowed batch files found in {data_dir}")

        # Metadata storage (lightweight)
        self.index_map = []      # List of tuples: (file_index, sample_index_in_file)
        self.bmi_values = []     # Store only necessary metadata in RAM
        self.bmi_labels = []
        self.subject_ids = []

        print("=" * 70)
        print(f"Indexing {len(self.batch_files)} batch files (Lazy Loading)...")
        print("=" * 70 + "\n")

        total_windows = 0

        # First pass: Load ONLY metadata (Demographics/IDs) to build the index
        for file_idx, batch_file in enumerate(self.batch_files):
            try:
                # Load only metadata headers, not the heavy PPG signals
                with np.load(batch_file, allow_pickle=True) as data:
                    # Extract lightweight metadata
                    demographics = data['Demographics']
                    sub_ids = data['SubjectIDs']
                    n_samples = len(sub_ids)

                    # Extract BMI info
                    # Assuming col 4 is value, col 5 is code based on your original code
                    batch_bmi_vals = demographics[:, 4].astype(np.float32)
                    batch_bmi_codes = demographics[:, 5].astype(np.float32)

                    # Filter valid samples immediately if requested
                    if remove_missing_bmi:
                        valid_mask = ~(np.isnan(batch_bmi_codes) | (batch_bmi_codes < 0))

                        # Apply mask
                        batch_bmi_vals = batch_bmi_vals[valid_mask]
                        batch_bmi_codes = batch_bmi_codes[valid_mask]
                        sub_ids = sub_ids[valid_mask]

                        # Store valid indices for this file
                        valid_indices = np.where(valid_mask)[0]
                        for valid_idx in valid_indices:
                            self.index_map.append((file_idx, valid_idx))
                    else:
                        # Store all indices
                        for i in range(n_samples):
                            self.index_map.append((file_idx, i))

                    self.bmi_values.append(batch_bmi_vals)
                    self.bmi_labels.append(batch_bmi_codes)
                    self.subject_ids.append(sub_ids)

                    total_windows += len(batch_bmi_vals)

                if (file_idx + 1) % 5 == 0:
                    print(f"Indexed {file_idx + 1}/{len(self.batch_files)} files...")

            except Exception as e:
                print(f"Error indexing {batch_file}: {str(e)}")
                raise

        # Concatenate metadata lists into arrays (much smaller than PPG data)
        self.bmi_values = np.concatenate(self.bmi_values, axis=0)
        self.bmi_labels = np.concatenate(self.bmi_labels, axis=0).astype(np.int64)
        self.subject_ids = np.concatenate(self.subject_ids, axis=0)

        # Setup class mappings
        unique_categories = np.unique(self.bmi_labels)
        self.num_classes = len(unique_categories)
        self.label_to_contiguous = {int(l): i for i, l in enumerate(sorted(unique_categories))}
        self.contiguous_to_label = {i: int(l) for l, i in self.label_to_contiguous.items()}

        # Create contiguous labels for training
        self.bmi_labels_contiguous = np.array(
            [self.label_to_contiguous[int(l)] for l in self.bmi_labels]
        )

        self.bmi_category_names = {
            0: "Underweight (< 18.5)",
            1: "Normal Weight (18.5-24.9)",
            2: "Overweight (25.0-29.9)",
            3: "Obese (≥ 30.0)",
        }

        print(f"\n✅ Total valid windows indexed: {len(self.index_map):,}")
        print(f"✅ Metadata loaded into RAM. PPG signals will be loaded on-demand.")

    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        # Retrieve file index and sample index from our map
        file_idx, sample_idx = self.index_map[idx]
        file_path = self.batch_files[file_idx]

        #change to .npy file
        npy_path = file_path.replace('.npz', '.npy')

        # Load specific window using memory mapping
        # mmap_mode='r' allows accessing parts of the array without loading whole file
        try:
            ppg_data = np.load(npy_path, mmap_mode='r')
            ppg_window = ppg_data[sample_idx]  # Access only this specific row
            ppg_tensor = torch.from_numpy(ppg_window.copy()).float().unsqueeze(0) # (1, L)

            # Get pre-loaded metadata
            bmi_value = torch.tensor(self.bmi_values[idx], dtype=torch.float32)

            return ppg_tensor, bmi_value

        except Exception as e:
            print(f"Error retrieving item {idx} from {file_path}: {e}")
            raise


In [ ]:
def create_subject_level_stratified_splits(dataset, test_size=0.2, random_state=42):
    """
    Create stratified train/val splits at SUBJECT LEVEL.
    Each subject's windows are all assigned to train OR val.
    """
    print(f"{'='*70}")
    print("Creating SUBJECT-LEVEL Stratified Split (No Data Leakage)")
    print(f"{'='*70}\n")

    unique_subjects = np.unique(dataset.subject_ids)
    n_subjects = len(unique_subjects)

    print(f"Total subjects: {n_subjects}")
    print(f"Total windows: {len(dataset)}")
    print(f"Avg windows per subject: {len(dataset) / n_subjects:.1f}\n")

    subject_to_indices = {}
    subject_to_bmi = {}

    for subj in unique_subjects:
        mask = (dataset.subject_ids == subj)
        indices = np.where(mask)[0]
        bmi_labels = dataset.bmi_labels_contiguous[indices]
        majority_bmi = np.bincount(bmi_labels).argmax()

        subject_to_indices[subj] = indices
        subject_to_bmi[subj] = majority_bmi

    subject_list = list(unique_subjects)
    bmi_labels_for_split = np.array([subject_to_bmi[s] for s in subject_list])

    print("Subject-level BMI distribution:")
    unique_bmi, counts = np.unique(bmi_labels_for_split, return_counts=True)
    for bmi_idx, count in zip(unique_bmi, counts):
        original_label = dataset.contiguous_to_label[bmi_idx]
        category_name = dataset.bmi_category_names.get(bmi_idx, "Unknown")
        print(f"  Class {bmi_idx} ({category_name}): {count} subjects ({count/n_subjects*100:.2f}%)")

    splitter = StratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )
    train_idx, val_idx = next(splitter.split(np.arange(n_subjects), bmi_labels_for_split))

    train_subjects = [subject_list[i] for i in train_idx]
    val_subjects = [subject_list[i] for i in val_idx]

    train_indices = np.concatenate([subject_to_indices[s] for s in train_subjects])
    val_indices = np.concatenate([subject_to_indices[s] for s in val_subjects])

    assert set(train_subjects).isdisjoint(set(val_subjects)), \
        "Subject overlap detected! Data leakage!"

    print(f"\n✅ Subject-level split completed (NO DATA LEAKAGE):")
    print(f"  Train subjects: {len(train_subjects)} ({len(train_subjects)/n_subjects*100:.1f}%)")
    print(f"  Val subjects: {len(val_subjects)} ({len(val_subjects)/n_subjects*100:.1f}%)")
    print(f"  Train windows: {len(train_indices):,} ({len(train_indices)/len(dataset)*100:.1f}%)")
    print(f"  Val windows: {len(val_indices):,} ({len(val_indices)/len(dataset)*100:.1f}%)\n")

    train_set = Subset(dataset, train_indices)
    val_set = Subset(dataset, val_indices)

    train_labels = dataset.bmi_labels_contiguous[train_indices]
    val_labels = dataset.bmi_labels_contiguous[val_indices]

    print(f"  Train set BMI distribution (windows):")
    unique_train, counts_train = np.unique(train_labels, return_counts=True)
    for label_idx, count in zip(unique_train, counts_train):
        original_label = dataset.contiguous_to_label[label_idx]
        category_name = dataset.bmi_category_names.get(label_idx, "Unknown")
        print(f"    Class {label_idx} ({category_name}): {count:,} windows ({count/len(train_labels)*100:.2f}%)")

    print(f"\n  Val set BMI distribution (windows):")
    unique_val, counts_val = np.unique(val_labels, return_counts=True)
    for label_idx, count in zip(unique_val, counts_val):
        original_label = dataset.contiguous_to_label[label_idx]
        category_name = dataset.bmi_category_names.get(label_idx, "Unknown")
        print(f"    Class {label_idx} ({category_name}): {count:,} windows ({count/len(val_labels)*100:.2f}%)")

    print(f"{'='*70}\n")

    return train_set, val_set

## create regressor model

In [ ]:
# 3 layers
class AnyPPGWithRegressor(nn.Module):
    def __init__(self, encoder, latent_dim, hidden_dims=(128, 64), dropout=0.3):
        super(AnyPPGWithRegressor, self).__init__()
        self.encoder = encoder
        self.regressor = nn.Sequential(
            nn.Linear(latent_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dims[1], 1)  # Single output for regression
        )

    def forward(self, x):
        latent = self.encoder(x)
        if latent.dim() > 2:
            latent = latent.reshape(latent.size(0), -1)
        bmi_pred = self.regressor(latent)
        return bmi_pred.squeeze(-1)  # Remove last dimension (batch_size,)


In [ ]:
from model.net1d import Net1D

# AnyPPG encoder configuration
anyppg_cfg = {
    "in_channels": 1,
    "base_filters": 64,
    "ratio": 1.0,
    "filter_list": [64, 160, 160, 400, 400, 1024],
    "m_blocks_list": [2, 2, 2, 3, 3, 1],
    "kernel_size": 3,
    "stride": 2,
    "groups_width": 16,
    "verbose": False,
}

# Initialize encoder
anyppg = Net1D(**anyppg_cfg)

# Load pretrained model weights
ckpt_path = "./checkpoint/anyppg_ckpt.pth"
state_dict = torch.load(ckpt_path, map_location="cpu")
anyppg.load_state_dict(state_dict)

In [ ]:
# with checkpoints saved:

def save_checkpoint(path, epoch, model, optimizer, scheduler, best_val_mae,
                    last_val_mae=None, last_val_rmse=None, train_losses=None, val_losses=None):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "best_val_mae": best_val_mae,
        "last_val_mae": last_val_mae,
        "last_val_rmse": last_val_rmse,
        "train_losses": train_losses if train_losses is not None else [],
        "val_losses": val_losses if val_losses is not None else [],
    }, path)


def load_checkpoint(path, device, model, optimizer=None, scheduler=None):
    ckpt = torch.load(path, map_location=device)
    model.load_state_dict(ckpt["model_state_dict"])

    if optimizer is not None and "optimizer_state_dict" in ckpt:
        optimizer.load_state_dict(ckpt["optimizer_state_dict"])

    if scheduler is not None and "scheduler_state_dict" in ckpt:
        scheduler.load_state_dict(ckpt["scheduler_state_dict"])

    start_epoch = ckpt.get("epoch", -1) + 1
    best_val_mae = ckpt.get("best_val_mae", float("inf"))
    train_losses = ckpt.get("train_losses", [])
    val_losses = ckpt.get("val_losses", [])

    return start_epoch, best_val_mae, train_losses, val_losses, ckpt


if __name__ == "__main__":
    print("\n" + "="*80)
    print("MAIN EXECUTION - BMI REGRESSION TRAINING (AnyPPG + Regression Head)")
    print("="*80)

    # CONFIGURATION
    windowed_dir = "/content/data_cache"
    save_dir = "/content/drive/MyDrive/CAS_FinalProject/Results/BMI_Regressor_AnyPPG_total"

    BATCH_SIZE = 1024
    NUM_EPOCHS = 10  # total epochs (not "additional")
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    CHECKPOINT_PATH = os.path.join(save_dir, "checkpoint_bmi_regressor_last.pth")
    BEST_MODEL_PATH = os.path.join(save_dir, "best_bmi_regressor.pth")

    os.makedirs(save_dir, exist_ok=True)

    # LOAD DATASET (use same dataset, but now regression targets)
    # (Assumes these are defined elsewhere in your codebase/notebook)
    full_dataset = PPGBMIDataset(windowed_dir, remove_missing_bmi=False)
    train_dataset, val_dataset = create_subject_level_stratified_splits(
        full_dataset, test_size=0.2, random_state=42
    )

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=0, pin_memory=True if DEVICE == "cuda" else False
    )
    val_loader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=0, pin_memory=True if DEVICE == "cuda" else False
    )

    # Initialize encoder (same as before)
    encoder = Net1D(**anyppg_cfg)
    anyppg_ckpt_path = "./checkpoint/anyppg_ckpt.pth"
    state_dict = torch.load(anyppg_ckpt_path, map_location="cpu")
    encoder.load_state_dict(state_dict)
    encoder.requires_grad_(False)

    # Define regression model
    latent_dim = 8192
    model_bmi_reg = AnyPPGWithRegressor(
        encoder=encoder,
        latent_dim=latent_dim,
        hidden_dims=(128, 64),
        dropout=0.3
    ).to(DEVICE)

    # Optimizer and MAE Loss
    optimizer = optim.Adam(model_bmi_reg.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.L1Loss()

    # LR scheduler (Reduce on plateau)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=5
    )

    # Resume if checkpoint exists
    start_epoch = 0
    best_val_mae = float("inf")
    train_losses, val_losses = [], []

    if os.path.isfile(CHECKPOINT_PATH):
        print(f"🔄 Resuming from: {CHECKPOINT_PATH}")
        start_epoch, best_val_mae, train_losses, val_losses, _ = load_checkpoint(
            CHECKPOINT_PATH, DEVICE, model_bmi_reg, optimizer, scheduler
        )
        print(f"   start_epoch={start_epoch}, best_val_mae={best_val_mae:.6f}")
    else:
        print("No checkpoint found. Starting from scratch.")

    # Training loop
    # Note: NUM_EPOCHS here is interpreted as TOTAL epochs.
    # If you want "train 5 more epochs", set: target_total_epochs = start_epoch + NUM_EPOCHS
    target_total_epochs = NUM_EPOCHS

    all_targets, all_preds = None, None
    val_rmse = None

    for epoch in range(start_epoch, target_total_epochs):
        # -------------------------
        # Training
        # -------------------------
        model_bmi_reg.train()
        train_loss = 0.0

        for ppg, bmi_target in tqdm(train_loader, desc=f"Training (epoch {epoch+1})"):
            ppg = ppg.to(DEVICE)
            bmi_target = bmi_target.to(DEVICE)

            optimizer.zero_grad(set_to_none=True)
            bmi_pred = model_bmi_reg(ppg)
            loss = criterion(bmi_pred, bmi_target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / max(1, len(train_loader))
        train_losses.append(avg_train_loss)

        # -------------------------
        # Validation
        # -------------------------
        model_bmi_reg.eval()
        val_loss = 0.0
        all_preds_list, all_targets_list = [], []

        with torch.no_grad():
            for ppg, bmi_target in tqdm(val_loader, desc=f"Validation (epoch {epoch+1})"):
                ppg = ppg.to(DEVICE)
                bmi_target = bmi_target.to(DEVICE)

                bmi_pred = model_bmi_reg(ppg)
                loss = criterion(bmi_pred, bmi_target)
                val_loss += loss.item()

                all_preds_list.append(bmi_pred.detach().cpu().numpy())
                all_targets_list.append(bmi_target.detach().cpu().numpy())

        avg_val_loss = val_loss / max(1, len(val_loader))
        val_losses.append(avg_val_loss)

        all_preds = np.concatenate(all_preds_list, axis=0)
        all_targets = np.concatenate(all_targets_list, axis=0)

        val_rmse = float(np.sqrt(np.mean((all_preds - all_targets) ** 2)))

        # Scheduler step on validation MAE
        scheduler.step(avg_val_loss)  # ReduceLROnPlateau expects the metric here. [web:32]

        print(
            f"Epoch {epoch+1}: "
            f"Train MAE: {avg_train_loss:.4f}, "
            f"Val MAE: {avg_val_loss:.4f}, "
            f"RMSE: {val_rmse:.2f}"
        )

        # -------------------------
        # Save BEST model
        # -------------------------
        if avg_val_loss < best_val_mae:
            best_val_mae = avg_val_loss
            torch.save({
                "epoch": epoch,
                "model_state_dict": model_bmi_reg.state_dict(),
                "val_mae": avg_val_loss,
                "val_rmse": val_rmse
            }, BEST_MODEL_PATH)
            print(f"  ✅ Best model saved! (Val MAE: {best_val_mae:.4f})")

        # -------------------------
        # Save LAST checkpoint (for resume)
        # -------------------------
        save_checkpoint(
            CHECKPOINT_PATH,
            epoch=epoch,
            model=model_bmi_reg,
            optimizer=optimizer,
            scheduler=scheduler,
            best_val_mae=best_val_mae,
            last_val_mae=avg_val_loss,
            last_val_rmse=val_rmse,
            train_losses=train_losses,
            val_losses=val_losses
        )

    # -------------------------
    # Plot losses
    # -------------------------
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train MAE", linewidth=2)
    plt.plot(val_losses, label="Val MAE", linewidth=2)
    plt.xlabel("Epoch")
    plt.ylabel("MAE Loss")
    plt.title(f"MAE Loss (Best Val: {best_val_mae:.4f})")
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    if all_targets is not None and all_preds is not None:
        plt.scatter(all_targets, all_preds, alpha=0.5, s=1)
        plt.plot([15, 40], [15, 40], "r--", linewidth=2, label="Perfect Prediction")
        plt.xlabel("True BMI")
        plt.ylabel("Predicted BMI")
        plt.title(f"Predictions vs Truth\nMAE: {best_val_mae:.4f}, RMSE: {val_rmse:.2f}")
        plt.legend()
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "bmi_regression_mae_results.png"), dpi=300, bbox_inches="tight")
    plt.close()
    print("MAE regression plots saved!")
